# [실습] 조건부 엣지와 툴 노드 연결하기  

이번 실습에서는 모델의 분기점을 구성하는 조건부 엣지와 외부 툴을 효과적으로 사용하는 툴 노드를 추가해 보겠습니다.

In [ ]:
!pip install langgraph langchain langchain_google_genai langchain_community

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens

    thinking_budget = 500  # 추론(Reasoning) 토큰 길이 제한
)

## 1. 조건부 엣지

조건부 엣지는 워크플로우의 분기점에 해당합니다.   
조건을 구성하여 종료 조건을 만들거나, 조건에 따라 다른 노드로 진입하도록 구성할 수 있습니다.

이번 실습에서는 멀티 턴의 대화를 구현해 보겠습니다.   
질문이 주어지면, 여러 번의 대화를 통해 맥락이 이어집니다.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


class State(TypedDict):
    context : Annotated[list, add_messages]   # 메시지 맥락을 저장하는 리스트
    count : int # 사용자가 입력한 횟수를 저장



In [ ]:
def ask_human(state):
    query = input()
    print('User :', query)

    return {'context':HumanMessage(content=query)}

def talk(state):
    messages = state['context']

    answer = llm.invoke(messages)

    print('AI :', answer.content)
    state['count'] +=1

    return {'context': answer, 'count': state['count']}


def check_end(state):

    return "Done" if state['count'] >= 3 else "Resume"
    # count가 3 이상이 되면 True, 이외 False






In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


builder = StateGraph(State)

builder.add_node('talk', talk)
builder.add_node('ask_human', ask_human)

builder.add_edge(START, 'ask_human'),
builder.add_edge('ask_human', 'talk'),
builder.add_conditional_edges('talk', check_end,
                              {'Done': END, 'Resume': 'ask_human'})


In [ ]:
graph = builder.compile()
graph

In [ ]:
system_prompt = '''한 문장 길이로만 대화하세요.'''

messages = [SystemMessage(content=system_prompt)]


response = graph.invoke({'context':messages, 'count':0})
response

## [연습문제] 종료 조건 수정하기

이번에는, 사용자가 'FINISHED'를 입력하면 대화가 끝나도록 만들어 보세요.   
check_end 함수의 내용과 그래프 구조를 수정하면 됩니다.

In [ ]:
def check_end(state):
    # 마지막 메시지를 찾고, 그 내용이 'FINISHED'면 끝
    return "Done" if state['context'][-1].content == 'FINISHED' else "Resume"

In [ ]:
builder = StateGraph(State)

builder.add_node('talk', talk)
builder.add_node('ask_human', ask_human)

builder.add_edge(START, 'ask_human'),
# builder.add_edge('ask_human', 'talk'),

builder.add_conditional_edges('ask_human', check_end,
                              {'Done': END, 'Resume': 'talk'})

builder.add_edge('talk', 'ask_human')

In [ ]:
graph = builder.compile()
graph

In [ ]:
system_prompt = '''한 문장 길이로만 대화하세요.'''
messages = [SystemMessage(content=system_prompt)]

graph.invoke({'context' :messages, 'count':0})

이번에는 초기 질문만 유저가 전달하면, LLM이 스스로 역할을 전환하며 답변하도록 만들어 보겠습니다.   
질문이 주어지면, 해당 질문으로 LLM이 User와 AI 역할을 반복하며 대화를 진행합니다.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


class State(TypedDict):
    context : list  # List Reducer를 쓰지 않는 이유? Context가 계속 바뀌기 때문
    count : int # 사용자가 입력한 횟수를 저장
    turn: str # 차례를 저장

In [ ]:
# 에이전트의 자동 발생 대화: 기존의 대화를 반대로 전환
def simulate(state):
    messages = state['context']
    switched_messages = []

    answer = llm.invoke(messages)
    messages.append(answer)


    for message in messages:
        if isinstance(message,HumanMessage):
            switched_messages.append(AIMessage(content=message.content))
        elif isinstance(message,AIMessage):
            switched_messages.append(HumanMessage(content=message.content))
        else:
            switched_messages.append(message)

    print(state['turn'], ':', messages[-1].content)

    if state['turn'] == 'AI':
        state['turn'] = 'User'
    else:
        state['turn'] = 'AI'

    state['count'] +=1

    return {'context':switched_messages, 'count':state['count'], 'turn' : state['turn']}

def check_end(state):
    return state['count'] >= 3





In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


builder = StateGraph(State)

builder.add_node('simulate', simulate)

builder.add_edge(START, 'simulate')
builder.add_conditional_edges('simulate', check_end,
                              {True: END, False: 'simulate'})


In [ ]:
graph = builder.compile()
graph

In [ ]:
system_prompt = '''대화는 무례하게 하세요. 욕설은 하지 마세요.'''
initial_question = input()

print('User :', initial_question)

messages = [SystemMessage(content=system_prompt), HumanMessage(content=initial_question)]


response = graph.invoke({'context':messages, 'count':0, 'turn':'AI'})
response


In [ ]:
messages = [SystemMessage(content=system_prompt), HumanMessage(content=initial_question)]

for chunk in graph.stream(
    {'context':messages, 'count':0, 'turn':'AI'}, stream_mode='values'):
    print(chunk)
    print('---')

간단한 종료조건을 만들었지만, 실제 코드에서는 'FINISHED' 등을 출력하게 하면 작업을 종료하는 프롬프트를 구성할 수도 있습니다.

## 2. Tool Node

툴 노드는 랭체인에서 지원하는 Prebuilt 계열의 코드로, 해당 노드로 툴 요청이 전달되면 그 결과를 실행합니다.   
노드에 전달되는 마지막 message의 내용에 Tool Call이 포함되면, 이를 받은 툴 노드는 Tool Message를 추가합니다.  

In [ ]:
# Tavily API
os.environ['TAVILY_API_KEY'] = 'tvly-xxxx'

툴을 정의합니다.

In [ ]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool


tavily_search = TavilySearch(
    max_results=3)

@tool
def multiply(x:int, y:int) -> int:
    "x와 y를 입력받아, x와 y를 곱한 결과를 반환합니다."
    return x*y

@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

print(multiply.invoke({'x':3, 'y':4}))
print(current_date.invoke({}))



In [ ]:
llm_with_tools = llm.bind_tools([multiply, current_date, tavily_search])

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


툴 요청 메시지를 출력할 노드를 구성합니다.

In [ ]:
def tool_calling_llm(state):
    return {"messages": llm_with_tools.invoke(state["messages"])}

In [ ]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


builder = StateGraph(State)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node('tools', ToolNode([multiply, current_date, tavily_search]))
# ToolNode: 입력이 전달되면 툴을 실행해서 돌려줌

builder.add_edge(START, 'tool_calling_llm')
builder.add_conditional_edges('tool_calling_llm', tools_condition, END)
# tools_condition: 툴이 필요하면 툴 노드로 이동, 아니면 END
builder.add_edge('tools', 'tool_calling_llm')


In [ ]:
graph = builder.compile()
graph

In [ ]:
response = graph.invoke({'messages':[
    HumanMessage(content='오늘 날짜에 태어난 배우는 누구야?')]})
response

In [ ]:
for data in graph.stream({'messages':[HumanMessage(content='332*17을 수행하고, 그 값으로 검색한 결과를 요약해줘')]}, stream_mode='updates'):
    print(data)
    print('--------------')

만약, 모델을 Gemini가 아닌, GPT-4.1-mini 같은 모델로 바꿔 본다면 어떻게 동작할까요?

In [ ]:
!pip install langchain_openai

In [ ]:
# OpenAI API 유료 크레딧이 있는 경우에만 실행 가능

os.environ['OPENAI_API_KEY'] = 'sk-...'

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4.1-mini')

In [ ]:
llm_with_tools = llm.bind_tools([multiply, current_date, tavily_search])

In [ ]:
def tool_calling_llm(state):
    return {"messages": llm_with_tools.invoke(state["messages"])}


builder = StateGraph(State)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node('tools', ToolNode([multiply, current_date, tavily_search]))

builder.add_edge(START, 'tool_calling_llm')
builder.add_conditional_edges('tool_calling_llm', tools_condition, END)
builder.add_edge('tools', 'tool_calling_llm')

graph = builder.compile()
graph

In [ ]:
response = graph.invoke({'messages':[
    HumanMessage(content='오늘 날짜에 태어난 배우는 누구야?')]})
response

In [ ]:
for data in graph.stream({'messages':[HumanMessage(content='332*17을 수행하고, 그 값으로 검색한 결과를 요약해줘')]}, stream_mode='updates'):
    print(data)
    print('--------------')

현재의 구조에서는, 모델에 따라 툴 실행 이행 능력과 일관성 있는 실행 능력의 차이가 발생합니다.   
특히, sLLM과 같은 모델의 경우 툴의 실행이 복잡하거나, Context가 길어지는 경우 오류가 발생할 수 있는데요.   
실습에서 다룬 것처럼, 프롬프트를 수정하여 작동 과정을 설명하거나, 예시를 프롬프트에 포함하여 방법을 보다 잘 이해하게 하는 방법을 고려합니다.